In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
from census import Census 
from us import states

from config import g_keypa
# Census API Key
from config import census_keypa
c = Census(census_keypa, year=2018)
# print(census_keypa)


In [6]:
# Run Census Search to retrieve data on all zip codes 
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B01002_002E",
                          "B01002_003E","B19301_001E"),
                           {'for': 'zip code tabulation area:*'})
                         
census_data
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B01002_002E": "Median Male Age",
                                      "B01002_003E": "Median Female Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",                                      
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Visualize
print(len(census_pd))
census_pd.head()
                         



33120


,Name,Household Income,Population,Median Age,Median Male Age,Median Female Age,Per Capita Income,Zipcode
0,ZCTA5 00601,13092.0,17242.0,40.5,39.5,41.3,6999.0,00601
1,ZCTA5 00602,16358.0,38442.0,42.3,41.9,42.8,9277.0,00602
2,ZCTA5 00603,16603.0,48814.0,41.1,39.1,42.9,11307.0,00603
3,ZCTA5 00606,12832.0,6437.0,43.3,43.2,43.4,5943.0,00606
4,ZCTA5 00610,19309.0,27073.0,42.1,40.4,44.2,10220.0,00610


In [20]:
# # Add in Income Rate (Per Capita Income / Population)
# census_pd["Income Rate"] = 100 * \
#     census_pd["Per Capita Income"].astype(int) / census_pd["Population"].astype(int)

# # Final DataFrame
# census_pd = census_pd[["Zipcode", "Population", "Median Age", "Median Male Age", "Median Female Age""Household Income",
#                        "Per Capita Income","Per Capita Income" ]]

# # Visualize, 
# print(len(census_pd))
# census_pd.head()

In [8]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [16]:
# Import the us zip data. Use dtype="object" to ensure all data is read in accurately.
us_zips = pd.read_csv("uszips.csv")

# Visualize
us_zips.head()

,Zipcode,lat,lng,city,state_id,state_name,zcta,parent_zcta
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN


In [11]:
# Merge the two data sets along zip code
census_data_complete = pd.merge(us_zips, census_pd, how="left", on=["Zipcode", "Zipcode"])

# # Remove rows missing data
# census_data_complete = census_data_complete.dropna()

# # Visualize
# census_data_complete.head()

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [18]:
#configure gmaps

gmaps.configure(api_key= g_keypa)

In [ ]:
# # Convert df to list
# df = census_data_complete["rate"].tolist()

In [19]:
# Create a map using uszips coordinates to set markers
marker_locations = us_zips[['lat', 'lng']].astype(float)

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig